In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

Test the assumption that all women survived and all men died:

In [4]:
female_survival = train_data.loc[train_data.Sex == "female"]["Survived"]
female_survival_rate = sum(female_survival) / len(female_survival)
print("Percentage of women who survived: " + str(100 * female_survival_rate)[:5] + "%")

male_survival = train_data.loc[train_data.Sex == "male"]["Survived"]
male_survival_rate = sum(male_survival) / len(male_survival)
print("Percentage of men who survived: " + str(100 * male_survival_rate)[:5] + "%")

Use a random forest model to predict survival status based on all other factors:

In [5]:
# Create random forest model
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

# Use these attributes to predict survival rates
features = ["Pclass", "Sex", "SibSp", "Parch"]
# Create subset of predictor data and turn categorical values into numerical values
# X for training predictor data, X_test for testing predictor data
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

# Create random forest classifier model and train model using training data X and Y
model = RandomForestClassifier(n_estimators = 100, max_depth = 5, random_state = 1)
model.fit(X, y)
# Predictions using test data, outputs to np.ndarray
predictions = model.predict(X_test)
# Convert to pd.DataFrame and save as csv
output = pd.DataFrame({"PassengerId" : test_data.PassengerId, "Survived" : predictions})
output.to_csv("submission.csv", index = False)
output.head()